# MODULES

In [19]:
import numpy as np
import pandas as pd
import math

import plotly.graph_objects as go

from scipy.stats import wrapcauchy
from scipy.stats import levy_stable
from scipy.spatial import distance

# CLASSES

In [2]:
################# http://www.pygame.org/wiki/2DVectorClass ##################
class Vec2d(object):
    """2d vector class, supports vector and scalar operators,
       and also provides a bunch of high level functions
       """
    __slots__ = ['x', 'y']

    def __init__(self, x_or_pair, y = None):
        if y == None:            
            self.x = x_or_pair[0]
            self.y = x_or_pair[1]
        else:
            self.x = x_or_pair
            self.y = y
            
    # Addition
    def __add__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x + other.x, self.y + other.y)
        elif hasattr(other, "__getitem__"):
            return Vec2d(self.x + other[0], self.y + other[1])
        else:
            return Vec2d(self.x + other, self.y + other)

    # Subtraction
    def __sub__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x - other.x, self.y - other.y)
        elif (hasattr(other, "__getitem__")):
            return Vec2d(self.x - other[0], self.y - other[1])
        else:
            return Vec2d(self.x - other, self.y - other)
    
    # Vector length
    def get_length(self):
        return math.sqrt(self.x**2 + self.y**2)
    
    # rotate vector
    def rotated(self, angle):        
        cos = math.cos(angle)
        sin = math.sin(angle)
        x = self.x*cos - self.y*sin
        y = self.x*sin + self.y*cos
        return Vec2d(x, y)

    
    # Set Vector length
    def __setlength(self, value):
        length = self.get_length()
        self.x *= value/length
        self.y *= value/length
        length = property(self.get_length, self.setlength, None, "gets or sets the magnitude of the vector")

# FUNTIONS

In [13]:
def bm_2d(n_steps = 1000, speed = 6, s_pos = [0,0]):
  """
  Arguments:
    n_steps:
    speed:
    s_pos:
  Return:
    BM_2d_df
  """
  # Init velocity vector
  velocity = Vec2d(speed,0)

  BM_2d_df = pd.DataFrame(columns = ['x_pos', 'y_pos'])
  temp_df = pd.DataFrame([{'x_pos': s_pos[0], 'y_pos': s_pos[1]}])

  BM_2d_df = pd.concat([BM_2d_df, temp_df], ignore_index=True)

  for i in range(n_steps-1):
    turn_angle = np.random.uniform(low=-np.pi, high=np.pi)
    velocity = velocity.rotated(turn_angle)

    temp_df = pd.DataFrame([{'x_pos': BM_2d_df.x_pos[i]+velocity.x, 'y_pos': BM_2d_df.y_pos[i]+velocity.y}])
    BM_2d_df = pd.concat([BM_2d_df, temp_df], ignore_index=True)
  return BM_2d_df

In [15]:
BM_other = bm_2d(1000,6,[3,9])

In [25]:
# Plotly go
fig_BM_2d = go.Figure()

# Plot trajectory
fig_BM_2d.add_trace(go.Scatter(
    x = BM_other.x_pos,
    y = BM_other.y_pos,
    marker = dict(size=2),
    line = dict(width=1),
    mode = 'lines',
    name = 'BM_2d',
    showlegend = True
))

fig_BM_2d.show()

# METRICS

## Path length

In [33]:
# BM speed = 3
BM_2d_df_3 = bm_2d(1000, 3, [0,0])

# BM speed = 6
BM_2d_df_6 = bm_2d(1000, 6, [0,0])

# CRW speed = 6
CRW_2d_df_9 = pd.read_csv('trajectories/crw_6_9.csv')

In [34]:
# PL BM whit speed 3
dis_BM_3 = np.array([distance.euclidean(BM_2d_df_3.loc[i-1], BM_2d_df_3.iloc[i]) for i in range(1,BM_2d_df_3.shape[0])])
pl_BM_3 = np.cumsum(dis_BM_3)

# PL BM whit speed 6
dis_BM_6 = np.array([distance.euclidean(BM_2d_df_6.loc[i-1], BM_2d_df_6.iloc[i]) for i in range(1,BM_2d_df_6.shape[0])])
pl_BM_6 = np.cumsum(dis_BM_6)

# PL CRW whit speed 6
dis_CRW_6 = np.array([distance.euclidean(CRW_2d_df_9.loc[i-1], CRW_2d_df_9.iloc[i]) for i in range(1,CRW_2d_df_9.shape[0])])
pl_CRW_6 = np.cumsum(dis_CRW_6)

In [35]:
# Plotly go
fig_path_length = go.Figure()

# Plot BM_3
fig_path_length.add_trace(go.Scatter(
    x = np.arange(len(pl_BM_3)),
    y = pl_BM_3,
    marker = dict(size=2),
    line = dict(width=2),
    mode = 'lines',
    name = 'path_length_BM_3',
    showlegend = True
))

# Plot BM_6
fig_path_length.add_trace(go.Scatter(
    x = np.arange(len(pl_BM_6)),
    y = pl_BM_6,
    marker = dict(size=2),
    line = dict(width=6),
    mode = 'lines',
    name = 'path_length_BM_6',
    showlegend = True
))

# Plot CRW_6
fig_path_length.add_trace(go.Scatter(
    x = np.arange(len(pl_CRW_6)),
    y = pl_CRW_6,
    marker = dict(size=2),
    line = dict(width=2),
    mode = 'lines',
    name = 'path_length_CRW_6',
    showlegend = True
))

fig_path_length.show()

In [45]:
BM_2d_df_6.y_pos

0               0
1        4.874132
2       -0.092441
3       -5.132982
4       -0.283891
          ...    
995   -142.504705
996    -137.65063
997   -137.095379
998   -140.481574
999   -135.593096
Name: y_pos, Length: 1000, dtype: object

In [76]:
# Plotly go
fig_3d = go.Figure()

# Plot trajectory
fig_3d.add_trace(go.Scatter3d(
    x = BM_2d_df_6.x_pos,
    y = BM_2d_df_6.y_pos,
    z = BM_2d_df_6.index,
    marker = dict(size=2),
    line = dict(color='blue', width=1),
    mode = 'lines',
    name = 'BM_2d_trajectory',
    showlegend = True
))

# Plot trajectory
fig_3d.add_trace(go.Scatter3d(
    x = CRW_2d_df_9.x_pos,
    y = CRW_2d_df_9.y_pos,
    z = CRW_2d_df_9.index,
    marker = dict(size=2),
    line = dict(color='red', width=2),
    mode = 'lines',
    name = 'CRW_2d_trajectory',
    showlegend = True
))

fig_3d.show()

## Mean Squared Displacement

In [47]:
# BM speed = 6
BM_2d_df_6 = pd.read_csv('trajectories/brownian_6.csv')

# CRW speed = 6
CRW_2d_df_6 = pd.read_csv('trajectories/crw_6_9.csv')

In [48]:
BM_2d_df_6.iloc[0:3]

,x_pos,y_pos
0,2.00000,5.000000
1,4.81113,-0.300712
2,5.24813,5.683353


In [49]:
distance.euclidean(BM_2d_df_6.iloc[0],BM_2d_df_6.iloc[2])

3.319234567852984

In [53]:
# Plotly go
fig_2d = go.Figure()

# Plot trajectory
fig_2d.add_trace(go.Scatter(
    x = BM_2d_df_6.iloc[0:3].x_pos,
    y = BM_2d_df_6.iloc[0:3].y_pos,
    marker = dict(size=2),
    line = dict(width=1),
    mode = 'lines',
    name = 'BM_2d',
    showlegend = True
))

fig_2d.show()

In [61]:
n = 100

# i = 3
displacement_vec = np.array([distance.euclidean(BM_2d_df_6.iloc[i-n], BM_2d_df_6.iloc[i]) 
                  for i in range(n, BM_2d_df_6.shape[0],1)])

In [71]:
met_df_2 = pd.read_csv('metrics/met_df_2.csv')
met_df_2

,MSD_BM,MSD_CRW
0,36.000000,3.600000e+01
1,74.422372,1.368477e+02
2,112.979349,2.966932e+02
3,154.127854,5.100141e+02
4,193.378233,7.720513e+02
...,...,...
994,98997.081835,2.512414e+06
995,97496.121246,2.520607e+06
996,96630.627308,2.529127e+06
997,97027.421373,2.539040e+06


In [74]:
# Init figure
fig_MSD = go.Figure()

# Firs BM_6
fig_MSD.add_trace(go.Scatter(
    x = met_df_2.index,
    y = met_df_2.MSD_CRW,
    marker = dict(size=2),
    line = dict(width=1),
    mode = 'lines',
    name = 'BM_2d',
    showlegend = True
))

# Firs BM_6
fig_MSD.add_trace(go.Scatter(
    x = met_df_2.index,
    y = met_df_2.MSD_BM,
    marker = dict(size=2),
    line = dict(width=1),
    mode = 'lines',
    name = 'BM_2d',
    showlegend = True
))

fig_MSD.show()

## Turning-angle distribution

In [ ]:
BM_2d_df_6_short = BM_2d_df_6.iloc[20:25]
BM_2d_df_6_short

In [68]:
# Init figure
fig_2d = go.Figure()

# Firs BM_6
fig_2d.add_trace(go.Scatter(
    x = BM_2d_df_6_short.x_pos,
    y = BM_2d_df_6_short.y_pos,
    marker = dict(size=2),
    line = dict(width=1),
    mode = 'lines',
    name = 'BM_2d',
    showlegend = True
))

fig_2d.show()

## Step-length distribution

In [78]:
Levy_2d_df_1 = pd.read_csv('trajectories/levy_6_1.csv')

Levy_2d_df_7 = pd.read_csv('trajectories/levy_6_7.csv')

In [ ]:
Levy_2d_df_1.info

In [80]:
# Plotly go
fig_3d = go.Figure()

# Plot trajectory
fig_3d.add_trace(go.Scatter3d(
    x = Levy_2d_df_1.x_pos,
    y = Levy_2d_df_1.y_pos,
    z = Levy_2d_df_1.index,
    marker = dict(size=2),
    line = dict(color='blue', width=1),
    mode = 'lines',
    name = 'Levy_2d_7_trajectory alpha=1.0, beta=1.0',
    showlegend = True
))

# Plot trajectory
fig_3d.add_trace(go.Scatter3d(
    x = Levy_2d_df_7.x_pos,
    y = Levy_2d_df_7.y_pos,
    z = Levy_2d_df_7.index,
    marker = dict(size=2),
    line = dict(color='red', width=2),
    mode = 'lines',
    name = 'Levy_2d_7_trajectory alpha=0.7, beta=1.0',
    showlegend = True
))

fig_3d.show()

Output hidden; open in https://colab.research.google.com to view.

In [82]:
fig_met_df = go.Figure()

loc = 3.0
beta = 1.0
resolution = 400
aux_domain = np.linspace(0, 30, resolution)

for Levy_exponent in [1.0, 0.7]:
  Levy_pdf = np.array([levy_stable.pdf(i, Levy_exponent, beta, loc) for i in aux_domain])

  # Firs BM_6
  fig_met_df.add_trace(go.Scatter(
      x = aux_domain,
      y = Levy_pdf,
      marker = dict(size=2),
      line = dict(width=2),
      mode = 'lines',
      name = 'Levy_{}'.format(Levy_exponent),
      showlegend = True
  ))

fig_met_df.show()

## Levy flight

In [85]:
# Init parameters
alpha = 1
beta = 1
stdMotionSteps = 6

speed = 1
n_samples = 100000

s_pos = {'x': 0, 'y': 0}

# Init velocity vector
velocity = Vec2d(speed, 0)

# Init DF
LW_2d_df = pd.DataFrame([{'x_pos': s_pos['x'], 'y_pos': s_pos['y']}])

i = 1
while i < n_samples:
  # Get ramdom n_steps from Levy distribution
  step_size = levy_stable.rvs(alpha, beta, stdMotionSteps)

  # Round to integer number
  step_size = int(np.ceil(abs(step_size)))

  theta = wrapcauchy.rvs(c=0.7, loc=0)

  # Uptade velocity
  velocity = velocity.rotated(theta)

  for j in range(step_size):
    temp_df = pd.DataFrame([{
        'x_pos': LW_2d_df.x_pos[i-1] + velocity.x,
        'y_pos': LW_2d_df.y_pos[i-1] + velocity.y
    }])

    # Add to the end to Levy's DF
    LW_2d_df = pd.concat([LW_2d_df, temp_df], ignore_index=True)
    i += 1

In [86]:
# Init figure
fig_Levy = go.Figure()

# Firs BM_6
fig_2d.add_trace(go.Scatter(
    x = LW_2d_df.x_pos,
    y = LW_2d_df.y_pos,
    marker = dict(size=2),
    line = dict(color='red', width=1),
    mode = 'lines',
    name = 'alpha{}, beta{},'.format(alpha, beta),
    showlegend = True
))

fig_Levy.show()

In [87]:
# Plotly go
fig_3d = go.Figure()

# Plot trajectory
fig_3d.add_trace(go.Scatter3d(
    x = LW_2d_df.x_pos,
    y = LW_2d_df.y_pos,
    z = LW_2d_df.index,
    marker = dict(size=2),
    line = dict(color='blue', width=1),
    mode = 'lines',
    name = 'Levy_2d_7_trajectory alpha=1.0, beta=1.0',
    showlegend = True
))


fig_3d.show()

Output hidden; open in https://colab.research.google.com to view.